# What is an LLM?
### Definition: A large language model (LLM) is a deep learning model trained on a vast amount of textual data, designed to understand and generate human-like language.
<img src="./What_is_llm.png" width="700" height="600">
<hr>
<br>

# Why Can’t We Run LLMs Directly on Consumer Hardware?
### Resource Requirements: The massive size (billions of parameters), memory constraints, and compute power needed for inference and training.
### Performance Bottlenecks: GPU/CPU limitations, latency, and cost factors.
### Example: Challenges of running models like GPT-3 on standard devices.
<img src="./llm_hardware.webp" width="700" height="600">
<hr>
<br>

# What is Ollama?
### Introduction to Ollama: A platform designed to efficiently run large language models on consumer-grade hardware.
### Key Features: Support for quantization, optimized inference, and simplified deployment workflows.

<img src="./what_is_ollama.webp" width="700" height="600">
<hr>
<br>

# How Does Ollama Make Running LLMs Easier? The concept of Quantization
### Optimizations: Memory management, efficient parallelization, and reduced latency.
<img src="./quantization.png" width="700" height="600">


# Getting Started with Ollama on macOS

Welcome to the "Getting Started with Ollama" notebook! In this notebook, we'll cover the installation, setup, and basic usage of Ollama, a powerful tool for deploying large language models on macOS.

## Table of Contents
1. [Prerequisites](#prerequisites)
2. [Installation](#installation)
3. [Basic Usage](#basic-usage)
4. [Conclusion](#conclusion)

---

## Prerequisites

Before you begin, ensure you have the following:

- **Python 3.7 or higher** installed on your machine.
- A working **terminal** application (you can use the built-in Terminal app).

To check your Python version, run the following command in your terminal:

```bash
python3 --version


## Installation

To install Ollama on macOS, you can use Homebrew, a popular package manager.

### Step 1: Install Homebrew (if not already installed)

If you haven't installed Homebrew yet, run the following command:


In [ ]:
!/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### Step 2: Install Ollama
Once Homebrew is installed, run the following command to install Ollama:

In [ ]:
!brew install ollama

### Step 3: Verify the Installation
To check if Ollama is installed correctly, run:

In [ ]:
!ollama --version

### Step 2: Run the Model
After pulling the model, you can run it with:

## Basic Usage

Now that you have Ollama installed, let's explore some basic commands.

### Step 1: Pull a Model

You can view a list of all pre-trained models available [here](https://ollama.com/library) but make sure to verify the RAM requirements.


You can pull a pre-trained model Llama 3.2 (3B parameters and requires 2GB RAM) using the following command:

In [ ]:
!ollama pull llama3.2

In [ ]:
!ollama run llama3.2

You can interact with the model through the terminal.


Try asking it a question or giving it a prompt!

# Demo 1: Automating Jira Tickets Creation 

### Step 1: Import necessary libraries

In [ ]:
import json
import ollama
import requests
from requests.auth import HTTPBasicAuth

### Step 2: Obtain your meeting transcription 

In [ ]:
with open("transcription.txt", "r") as f:
    transcription = f.read()

### Step 3: Extract tasks from Ollama

In [ ]:

PROMPT = """Please analyze the following transcription of a conversation between different team members and extract actionable items or tasks. Return the extracted tasks in JSON format only, without any additional text or commentary. Each task should include a summary, a description, and the name of the assignee.

Transcription: {transcription}

Expected JSON Response:

[
    {{
        "summary": "Task summary here",
        "description": "Detailed description of the task",
        "assignee": "Name of the assignee"
    }},
    {{
        "summary": "Another task summary here",
        "description": "Detailed description of the second task",
        "assignee": "Name of the second assignee"
    }}
]

Make sure that the JSON response contains all relevant tasks discussed in the conversation and no other text.
"""

In [ ]:
response = ollama.chat(model='llama3.2:latest', messages=[
  {
    'role': 'user',
    'content': PROMPT.format(transcription=transcription),
  },],
    stream=True)

final_response = []
for chunk in response:
    print(chunk['message']['content'], end="")
    final_response.append(chunk['message']['content'])


In [ ]:
final_response = "".join(final_response)

try:
    tasks = json.loads(final_response)
except json.JSONDecodeError:
    print("Invalid JSON response")
    exit(1)

### Step 4: Get Jira Project Details
[Jira REST API Documentation](https://developer.atlassian.com/cloud/jira/platform/rest/v3/intro/#version)


[Get API Token](https://id.atlassian.com/manage-profile/security/api-tokens)

In [ ]:
# Define the necessary variables
JIRA_URL = ""

EMAIL = ""
API_TOKEN = ""

# You need any one of the following
PROJECT_KEY = "SCRUM"
PROJECT_ID = "10000"

# Account IDs of your team members (use this query: https://beinghasnain16.atlassian.net/rest/api/2/users/search?query)
ASSIGNEES = {"Hasnain": "",
             "Zohaib": ""}

# Define the Issue type and Attributes
ISSUE_DATA= { "fields": {
        "project": {
            "id": PROJECT_ID,
        },
        "summary": "",
        "description": "",
        "issuetype": {
            "name": "Task"
        },
        "assignee": {
            "id": ""
        },
        }
}

### Step 5: Create Tasks

In [ ]:
# Make the request to create the issue
for task in tasks:
    issue_data = ISSUE_DATA.copy()
    issue_data["fields"]['summary'] = task["summary"]
    issue_data["fields"]['description'] = task["description"]
    issue_data["fields"]['assignee']['id'] = ASSIGNEES.get(task["assignee"], "")

    response = requests.post(
        JIRA_URL,
        data=json.dumps(issue_data),
        headers={"Content-Type": "application/json"},
        auth=HTTPBasicAuth(EMAIL, API_TOKEN)
    )

    # Check the response
    if response.status_code == 201:
        print("Task created successfully:", response.json())
    else:
        print("Failed to create task:", response.status_code, response.text)


## Thank you

# Demo 2: Imitating the Apple AI - Building a simple local chatgpt available on all applications

## Step 1: Installing the required libraries

In [ ]:
!pip install ollama pyperclip pynput

## Step 2: Importing the required libraries
### Description of installed libraries
#### ollama: Ollama's Python sdk to deal with locally running ollama
#### pyperclip: Deals with the laptop's clipboard. Copy to clipboard, Paste from clipboard etc
#### pynput: Used to control the keyboard inputs and use copy pasting shortcuts

In [ ]:
!pip show pyperclip

In [1]:
import time
import pyperclip
import ollama

from string import Template
from pynput import keyboard
from pynput.keyboard import Key, Controller

## Step 3: Initializing the Controller (to control keyboard input) 

In [2]:
controller = Controller()

## Step 4: Defining the templates that will be used later by the llm

In [3]:
PROMPT_TEMPLATE = Template(
    """Fix all typos and casing and punctuation in this text, but preserve all new line characters:

$text

Return only the corrected text, don't include a preamble.
"""
)

PROMPT_TEMPLATE_PERSONALIZED = Template(
    """$prompt:$text"""
)

PROMPT_TEMPLATE_CODING_ASSISTANT = Template(
    """
    You are a helpful coding assistant who can code in any language. Below is a code snippet or an instruction for a code. If there's any error in the code fix it. If its just a comment for a code
    Complete the function. Dont add anything extra, just write the code as if you were in a code editor. Dont add anything other than the code or comments. For comments use appropriate syntax for the language. Dont add an inverted commas for anything
    $text
    Here is an additional instruction $prompt
    . dont even add the (''') in the start and end. Just only give the code and NOTHING else"""
)

## Step 5: Function to call Ollama endpoint and stream the response from LLM

In [4]:
def get_response(prompt):
    stream = ollama.chat(
        model='llama3.2:1b',
        messages=[{'role': 'user', 'content': prompt}],
        stream=True,
    )

    for chunk in stream:
        pyperclip.copy(chunk['message']['content'])
        time.sleep(0.1)
        print(chunk['message']['content'], end='', flush=True)
        with controller.pressed(Key.cmd):
            controller.tap("v")
        time.sleep(0.1)

## Step 6: Different functions to handle different kind of commands

In [5]:
def fix_text(text, prompt=None, type='text'):
    prompt = PROMPT_TEMPLATE_PERSONALIZED.substitute(text=text, prompt=prompt) if prompt and type!= "Code" else PROMPT_TEMPLATE_CODING_ASSISTANT.substitute(text=text, prompt=prompt) if type=="Code" else PROMPT_TEMPLATE.substitute(text=text)
    try:
        print("Prompt", prompt)
        get_response(prompt)
    except Exception as e:
        print("Error", e)
        return

def fix_current_line():
    # macOS short cut to select current line: Cmd+Shift+Left
    controller.press(Key.cmd)
    controller.press(Key.shift)
    controller.press(Key.left)

    controller.release(Key.cmd)
    controller.release(Key.shift)
    controller.release(Key.left)

    fix_selection()

def fix_selection():
    # 1. Copy selection to clipboard
    with controller.pressed(Key.cmd):
        controller.tap("c")

    # 2. Get the clipboard string
    time.sleep(0.1)
    text = pyperclip.paste()
    print("Copied text", text)

    # 3. Fix string
    if not text:
        return
    fix_text(text)

def fix_selection_with_prompt(type='text'):
    # 1. Copy selection to clipboard
    with controller.pressed(Key.cmd):
        controller.tap("c")

    # 2. Get the clipboard string
    time.sleep(0.1)
    text = pyperclip.paste()
    try:
        prompt_with_selection = text.split("..")
        prompt = prompt_with_selection[1]
        selected_text = prompt_with_selection[0]
    except:
        prompt = ""
        selected_text = text
    if not text:
        return
    print("Prompt", prompt)
    print("Selected text", selected_text)
    fix_text(selected_text, prompt, type)


## Step 7: Key Binding Functions

In [6]:
def on_f7():
    fix_current_line()

def on_f8():
    fix_selection()

def on_f9():
    fix_selection_with_prompt()

def on_f10():
    fix_selection_with_prompt('Code')


## Final Step: Continously running the assistant and binding different keys to different commands
### Before running the final cell, Make sure to allow some permissions for your code editor
#### Go to Settings -> Privacy and Security -> Accessibility and turn it on for your Code Editor
#### Then again Privacy and Security -> Input Monitoring and turn it on for your Code Editor
### These are the available commands
### F7 : Fix the current line. (Move the cursor to end of line)
### F8 : Fix the selected paragraph
### F9 : Give a prompt along with the highlighted text (Can be used to write anything)
### F10 : Coding Assistant with prompt

In [ ]:
with keyboard.GlobalHotKeys({"<100>": on_f7, "<98>": on_f8, "<101>": on_f9, "<109>": on_f10}) as h:
    h.join()

## Conclusion

In this notebook, we covered the installation and basic usage of Ollama on macOS. With the provided commands, you can easily set up and start using language models in your projects.

We also discussed how we can automate the Jira tickets creation process with LLMs.

For more information, visit the official [Ollama documentation](https://ollama.com/docs).

Happy coding!
